In [8]:
import torch 
from torch import nn

import numpy as np
from PIL import Image
import torch.optim as optim 

import torchvision
import torchvision.transforms as transforms
from torchvision import models 
from tqdm import tqdm
import os

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    # transforms.Resize(224,224)
])

In [10]:
train_data = torchvision.datasets.CIFAR100(root='data',download = True,transform = transform,train = True)
test_data = torchvision.datasets.CIFAR100(root='data',download = True,transform = transform,train = False)

In [29]:
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 64,
    shuffle=  True,
    num_workers = 2
)
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 64,
    shuffle=  True,
    num_workers = 2
)

In [12]:
image,label = train_data[1]
print(f"{image,label}\nimage shape = {image.shape}")

(tensor([[[1.0000, 0.9843, 0.9843,  ..., 0.9843, 0.9843, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 0.9843, 0.9843,  ..., 0.9843, 0.9843, 1.0000]],

        [[1.0000, 0.9843, 0.9843,  ..., 0.9843, 0.9843, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 0.9843, 0.9843,  ..., 0.9843, 0.9843, 1.0000]],

        [[1.0000, 0.9843, 0.9843,  ..., 0.9843, 0.9843, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1

In [24]:
# Load pretrained ResNet-18 model
resnet_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final layer to match CIFAR-10 (10 classes)
num_features = resnet_model.fc.in_features  # Get the input size of the last layer
resnet_model.fc = nn.Linear(num_features, 100)  # Replace with 10 output classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
class NeuralNet(nn.Module):
    def __init__(self,input_shape,output_shape):
        super().__init__()
        self.conv_1 = nn.Sequential(
            nn.Conv2d(in_channels = input_shape,
                     out_channels = 12,
                     kernel_size = 3,
                     stride = 1,
                     padding = 1), #output size = (12,32,32)
            nn.LeakyReLU(),
            nn.BatchNorm2d(12),
            nn.MaxPool2d(kernel_size = 2,
                        stride = 2) #output = (12,16,16)
        )
        self.conv_2 = nn.Sequential(
            nn.Conv2d(in_channels = 12,
                     out_channels = 24,
                     kernel_size = 3,
                     stride = 1,
                     padding = 1), #output size = (24,16,16)
            nn.LeakyReLU(),
            nn.BatchNorm2d(24),
            nn.MaxPool2d(kernel_size = 2,
                        stride = 2) # output size =(24,8,8)
        )
        self.conv_3 = nn.Sequential(
            nn.Conv2d(in_channels = 24,
                     out_channels = 48,
                     kernel_size = 3,
                     stride = 1,
                     padding = 1), #output size = (48,8,8)
            nn.LeakyReLU(),
            nn.BatchNorm2d(48),
            nn.MaxPool2d(kernel_size = 2,
                        stride = 2) # output size =(48,4,4)
        )
        self.conv_4 = nn.Sequential(
            nn.Conv2d(in_channels = 48,
                     out_channels = 96,
                     kernel_size = 3,
                     stride = 1,
                     padding = 1), #output size = (96,4,4)
            nn.LeakyReLU(),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(kernel_size = 2,
                        stride = 2) # output size =(96,2,2)
        )
        self.fc1 = nn.Linear( #fallten 
            96*2*2,120
        )
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,32)
        self.fc4 = nn.Linear(32,output_shape) #fc = fully connected
        self.dropout = nn.Dropout(0.3)
        self.leakyrelu = nn.LeakyReLU()

    def forward(self,x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)

        x = torch.flatten(x,1)
        
        x = self.leakyrelu(self.dropout(self.fc1(x)))
        x = self.leakyrelu(self.dropout(self.fc2(x)))
        x = self.leakyrelu(self.dropout(self.fc3(x)))
        x = self.dropout(self.fc4(x))         
        return x

In [16]:
self_model = NeuralNet(3,len(train_data.classes)).to(device)

In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer_self = optim.SGD(self_model.parameters(),lr = 0.001,momentum = 0.9)
optimizer_resnet = optim.SGD(resnet_model.parameters(),lr = 0.001,momentum = 0.9)

In [ ]:
start_epoch = 0
checkpoint_path = "Models/checkpoint.pth"
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    self_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer_self.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh.")

EPOCHS = 30
for epoch in tqdm(range(start_epoch,30)):
    print(f"Training epoch: {epoch}...")

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        
        input,labels = data
        input,labels = input.to(device),labels.to(device)
        optimizer_self.zero_grad()
        outputs = self_model(input)

        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer_self.step()
        running_loss += loss.item()

    checkpoint = {
            'epoch': epoch,
            'model_state_dict': self_model.state_dict(),
            'optimizer_state_dict': optimizer_self.state_dict(),
        }
    torch.save(checkpoint, checkpoint_path)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(train_loader):.4f}, Checkpoint saved!")

    print(f"loss: {running_loss/len(train_loader):.4f} ")

Resuming training from epoch 12


  0%|                                                                                           | 0/18 [00:00<?, ?it/s]

Training epoch: 12...


  6%|████▌                                                                              | 1/18 [00:21<06:03, 21.36s/it]

Epoch 13/30, Loss: 3.7273, Checkpoint saved!
loss: 3.7273 
Training epoch: 13...


 11%|█████████▏                                                                         | 2/18 [00:42<05:43, 21.50s/it]

Epoch 14/30, Loss: 3.7072, Checkpoint saved!
loss: 3.7072 
Training epoch: 14...


 17%|█████████████▊                                                                     | 3/18 [01:05<05:28, 21.92s/it]

Epoch 15/30, Loss: 3.6780, Checkpoint saved!
loss: 3.6780 
Training epoch: 15...


In [27]:
start_epoch = 0
checkpoint_path = "Models/checkpoint_resnet.pth"
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    resnet_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer_resnet.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh.")

EPOCHS = 30
for epoch in tqdm(range(start_epoch,EPOCHS)):
    print(f"Training epoch: {epoch}...")


    running_loss = 0.0
    for i, data in enumerate(train_loader):
        input,labels = data
        input,labels = input.to(device),labels.to(device)
        optimizer_resnet.zero_grad()
        outputs = resnet_model(input)

        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer_resnet.step()
        running_loss += loss.item()
    checkpoint = {
            'epoch': epoch,
            'model_state_dict': resnet_model.state_dict(),
            'optimizer_state_dict': optimizer_resnet.state_dict(),
        }
    torch.save(checkpoint, checkpoint_path)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(train_loader):.4f}, Checkpoint saved!")
    print(f"loss: {running_loss/len(train_loader):.4f} ")

No checkpoint found. Starting fresh.


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Training epoch: 0...


 10%|████████▎                                                                          | 1/10 [00:46<07:00, 46.69s/it]

Epoch 1/10, Loss: 4.6625, Checkpoint saved!
loss: 4.6625 
Training epoch: 1...


 20%|████████████████▌                                                                  | 2/10 [01:32<06:07, 45.98s/it]

Epoch 2/10, Loss: 4.5955, Checkpoint saved!
loss: 4.5955 
Training epoch: 2...


 20%|████████████████▌                                                                  | 2/10 [01:38<06:33, 49.14s/it]


KeyboardInterrupt: 

In [41]:
torch.save(self_model.state_dict(),'Models/trained_self.pth')
torch.save(resnet_model.state_dict(),'Models/trained_resnet.pth')

In [42]:
self_model = NeuralNet(3,len(train_data.classes)).to(device)
self_model.load_state_dict(torch.load('Models/trained_self.pth'))
resnet_model = resnet_model.to(device)
resnet_model.load_state_dict(torch.load('Models/trained_resnet.pth'))

<All keys matched successfully>

In [28]:
correct = 0
total = 0

self_model.eval()
with torch.inference_mode():
    for data in test_loader:
        images,labels = data
        images,labels = images.to(device),labels.to(device)
        outputs = self_model(images)
        _,predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100*correct/total
print(f"{accuracy}")

74.97


In [29]:
correct = 0
total = 0

resnet_model.eval()
with torch.inference_mode():
    for data in test_loader:
        images,labels = data
        images,labels = images.to(device),labels.to(device)
        outputs = resnet_model(images)
        _,predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100*correct/total
print(f"{accuracy}")

67.66


In [39]:
class_names=  ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['example.jpg','example_1.jpeg','example_2.jpg']
images = [load_image(img) for img in image_paths]

self_model.eval()
with torch.inference_mode():
    for image in images:
        image = image.to(device)
        output = self_model(image)
        _,predicted = torch.max(output,1)
        print(f"Prediction: {class_names[predicted.item()]}")

Prediction: plane
Prediction: deer
Prediction: dog


In [38]:
class_names=  ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['example.jpg','example_1.jpeg','example_2.jpg']
images = [load_image(img) for img in image_paths]

self_model.eval()
with torch.inference_mode():
    for image in images:
        image = image.to(device)
        output = resnet_model(image)
        _,predicted = torch.max(output,1)
        print(f"Prediction: {class_names[predicted.item()]}")

Prediction: plane
Prediction: horse
Prediction: bird
